In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="SymbolDatabase.GetPrototype() is deprecated")
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

import pickle
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
import threading
import time
import logging
from tkinter import messagebox

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Load the model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Initialize the video capture
cap = cv2.VideoCapture(0)

# Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, max_num_hands=1)

# Labels dictionary
labels_dict = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H',
    8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O',
    15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V',
    22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: '0', 27: '1', 28: '2',
    29: '3', 30: '4', 31: '5', 32: '6', 33: '7', 34: '8', 35: '9',
    36: 'I love You', 37: 'Yes', 38: 'No', 39: 'Hello', 40: 'Thanks',
    41: 'Sorry', 42: 'space'
}

# Predefined list of common words for auto-suggestions
common_words = ['HELLO', 'THANKS', 'YES', 'SORRY', 'LOVE']

# Create a tkinter window
root = tk.Tk()
root.title("ASL Prediction")

# Create a text field in tkinter
text_field = tk.Text(root, height=2, width=40, font=("Helvetica", 16))
text_field.pack()

# Create a clear text button
def clear_text():
    text_field.delete('1.0', tk.END)  # Clear the text field
    logging.info('Text cleared.')

clear_button = tk.Button(root, text="Clear Text", command=clear_text)
clear_button.pack()

# Variable to store the previous prediction and time
prev_prediction = None
word_count = 0  # Track how many words have been written

# Variables to track the detected character and delay counter
last_detected_character = None
fixed_character = ""
delayCounter = 0
start_time = time.time()

# Prompt Engineering Variables
current_word = ""
prompt_answer = None
word_history = []  # To keep track of detected words

# Function to display a confirmation prompt to the user
def prompt_user(predicted_word):
    global prompt_answer
    response = messagebox.askyesno("Word Confirmation", f"Is '{predicted_word}' the correct word?")
    if response:
        prompt_answer = "Yes"
        word_history.append(predicted_word)  # Add confirmed word to history
    else:
        prompt_answer = "No"
        auto_correct(predicted_word)  # Trigger auto-correction if the word is incorrect

# Function to auto-correct the detected word
def auto_correct(current_word):
    corrections = [word for word in common_words if word.startswith(current_word)]
    if corrections:
        suggestion = corrections[0]
        response = messagebox.askyesno("Auto-Correction", f"Did you mean: '{suggestion}'?")
        if response:
            logging.info(f"Auto-corrected to: {suggestion}")
            update_text_field(suggestion)
            return suggestion
    logging.info("No auto-correction found.")
    return current_word

# Function to display a character confirmation prompt for feedback
def prompt_character(character):
    response = messagebox.askyesno("Character Confirmation", f"Is '{character}' the correct character?")
    return response

# Function to show word history
def show_history():
    history_string = '\n'.join(word_history)
    messagebox.showinfo("Word History", f"Detected words:\n{history_string}")

# Function to update the tkinter text field by appending the new predicted character
def update_text_field(text):
    if text == 'space':
        text_field.insert(tk.END, ' ')  # Append a space
    else:
        text_field.insert(tk.END, text + '')  # Append new character
    logging.info(f'Word added: {text if text != "space" else "space (represented as space)"}')

# Function to auto-suggest words after detecting a few characters
def auto_suggest(word_fragment):
    suggestions = [word for word in common_words if word.startswith(word_fragment)]
    if suggestions:
        suggestion = suggestions[0]
        logging.info(f"Auto-suggested word: {suggestion}")
        return suggestion
    return None

# Function to run video capture and ASL prediction in a separate thread
def run():
    global last_detected_character, fixed_character, delayCounter, start_time, current_word, prompt_answer

    while True:
        data_aux = []
        x_ = []
        y_ = []

        ret, frame = cap.read()

        if not ret:
            break

        H, W, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

                x1 = int(min(x_) * W) - 10
                y1 = int(min(y_) * H) - 10
                x2 = int(max(x_) * W) - 10
                y2 = int(max(y_) * H) - 10

                # Make prediction using the model
                prediction = model.predict([np.asarray(data_aux)])
                predicted_character = labels_dict[int(prediction[0])]

                # Draw a rectangle and the predicted character on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                            cv2.LINE_AA)

                current_time = time.time()

                # Timer logic: Check if the predicted character is the same for more than 1 second
                if predicted_character == last_detected_character:
                    if (current_time - start_time) >= 1.0:  # Class fixed after 1 second
                        fixed_character = predicted_character
                        if delayCounter == 0:  # Add character once after it stabilizes for 1 second
                            if prompt_character(fixed_character):  # Ask for feedback on each character
                                update_text_field(fixed_character)
                                current_word += fixed_character

                                # Auto-suggest words after a few characters
                                if len(current_word) >= 3:  # Suggest after 3 characters
                                    suggestion = auto_suggest(current_word)
                                    if suggestion:
                                        prompt_user(suggestion)

                            delayCounter = 1

                            # Prompt the user for confirmation if the word is ready
                            if len(current_word) >= 5:  # Example condition for prompting after 5 characters
                                prompt_user(current_word)
                                if prompt_answer == "No":
                                    logging.info("Word incorrect. Retrying...")
                                    current_word = ""  # Reset word if wrong
                                else:
                                    logging.info(f"Word confirmed: {current_word}")
                                    current_word = ""  # Clear for next word
                else:
                    # Reset the timer when a new character is detected
                    start_time = current_time
                    last_detected_character = predicted_character
                    delayCounter = 0  # Reset delay counter for a new character

        # Show the video feed with the prediction
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to display word history
def display_history():
    show_history()
# Function to exit the application and stop both Tkinter and OpenCV
def exit_app():
    global cap
    logging.info('Exiting application...')
    if cap.isOpened():
        cap.release()  # Release the video capture
    cv2.destroyAllWindows()  # Close OpenCV windows
    root.quit()  # Stop the Tkinter main loop
    root.destroy()  # Close the Tkinter window

# Create an "Exit" button
exit_button = tk.Button(root, text="Exit", command=exit_app)
exit_button.pack()

# Thread to run the video capture and prediction
threading.Thread(target=run, daemon=True).start()

# Add a button to show word history
history_button = tk.Button(root, text="Show History", command=display_history)
history_button.pack()

root.mainloop()


2024-10-23 20:05:19,149 - Word added: Hello
2024-10-23 20:05:21,324 - Word confirmed: Hello
2024-10-23 20:05:24,726 - Word added: space (represented as space)
2024-10-23 20:05:25,947 - Word confirmed: space
2024-10-23 20:05:31,147 - Word added: I
2024-10-23 20:05:35,509 - Word added: A
2024-10-23 20:05:38,845 - Word added: M
2024-10-23 20:05:43,698 - Word added: space (represented as space)
2024-10-23 20:05:44,778 - Word confirmed: IAMspace
2024-10-23 20:05:48,463 - Word added: A
2024-10-23 20:05:51,568 - Word added: H
2024-10-23 20:05:54,311 - Word added: M
2024-10-23 20:05:57,601 - Word added: E
2024-10-23 20:06:02,067 - Word added: D
2024-10-23 20:06:03,093 - Word confirmed: AHMED
2024-10-23 20:06:05,072 - Word added: space (represented as space)
2024-10-23 20:06:06,286 - Word confirmed: space
2024-10-23 20:06:08,261 - Word added: 0
2024-10-23 20:06:11,177 - Word added: 1
2024-10-23 20:06:13,625 - Word added: 2
2024-10-23 20:06:17,492 - Word added: 3
2024-10-23 20:06:21,238 - Word a